In [2]:
import sys
print(sys.executable)

c:\Users\Beeclick\Documents\MIAAD\S3\Data Mining\Project\data-mining-project\DataMiningEnv\Scripts\python.exe


### Import des bibliothèques

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Chargement des données brutes

In [4]:
clients = pd.read_csv("../data/raw/Clients.csv")
catalogue = pd.read_csv("../data/raw/Catalogue.csv")
immatriculations = pd.read_csv("../data/raw/immatriculations_small.csv")
marketing = pd.read_csv("../data/raw/Marketing.csv")

# Nettoyage des données pour les clients

In [7]:
# Affichage des premières lignes
print("Aperçu des données brutes:")
clients.head()

Aperçu des données brutes:


,age,sexe,taux,situationFamiliale,nbEnfantsAcharge,2eme voiture,immatriculation
0,25,F,159,En Couple,2,false,3467 SB 72
1,53,M,594,En Couple,2,false,113 LY 42
2,20,F,949,En Couple,1,false,925 WK 87
3,29,M,571,En Couple,2,false,3279 RV 81
4,47,M,502,En Couple,1,false,82 RZ 54


In [13]:
# Remplacer les valeurs incorrectes (erreurs de saisie) par NaN
clients.replace(['?', 'N/D', ' '], pd.NA, inplace=True)

In [14]:
# Vérification des types de données et des valeurs manquantes
print("\nInformations sur les données:")
clients.info()
print("\nValeurs manquantes par colonne:")
clients.isnull().sum()


Informations sur les données:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   age                 99807 non-null   object
 1   sexe                99709 non-null   object
 2   taux                99806 non-null   object
 3   situationFamiliale  99681 non-null   object
 4   nbEnfantsAcharge    99800 non-null   object
 5   2eme voiture        99780 non-null   object
 6   immatriculation     100000 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB

Valeurs manquantes par colonne:


age                   193
sexe                  291
taux                  194
situationFamiliale    319
nbEnfantsAcharge      200
2eme voiture          220
immatriculation         0
dtype: int64

In [58]:
# Conversion en type numérique en remplaçant les valeurs non convertibles par NaN
clients['age'] =  pd.to_numeric(clients['age'], errors='coerce')
clients['taux'] = pd.to_numeric(clients['taux'], errors='coerce')
clients['nbEnfantsAcharge'] = pd.to_numeric(clients['nbEnfantsAcharge'], errors='coerce')

In [35]:
# Recherche des doublons
duplicates = clients.duplicated().sum()
print(f"\nNombre de doublons : {duplicates}")


Nombre de doublons : 0


In [11]:
# Suppression des doublons
# df = df.drop_duplicates()

In [29]:
# Compter les occurrences pour chaque colonne de type 'object'
for col in clients.columns:
    if clients[col].dtype == 'object':
        print(clients[col].value_counts())
        print()

# or
# clients['sexe'].unique() #...

sexe
M           68283
F           29388
Homme         743
Masculin      684
Féminin       322
Femme         289
Name: count, dtype: int64

situationFamiliale
En Couple      63855
Célibataire    29841
Seule           4989
Marié(e)         664
Seul             282
Divorcée          50
Name: count, dtype: int64

2eme voiture
false    86845
true     12935
Name: count, dtype: int64

immatriculation
835 JQ 59     2
4229 XL 63    2
7084 RU 46    2
3648 SM 17    2
3142 RV 53    2
             ..
5659 SO 55    1
6760 YM 11    1
385 UN 57     1
4056 HI 13    1
3467 SB 72    1
Name: count, Length: 99989, dtype: int64



In [30]:
# Remplacer les valeurs dans la colonne "sexe"
clients['sexe'] = clients['sexe'].replace({
    'Homme': 'M',
    'Masculin': 'M',
    'Féminin': 'F',
    'Femme': 'F',
})

In [31]:
# Remplacer les valeurs dans la colonne "sexe"
clients['situationFamiliale'] = clients['situationFamiliale'].replace({
    'Seule': 'Célibataire',
    'Seul': 'Célibataire',
})

In [34]:
clients.describe(include='object')

,sexe,situationFamiliale,2eme voiture,immatriculation
count,99709,99681,99780,100000
unique,2,4,2,99989
top,M,En Couple,false,835 JQ 59
freq,69710,63855,86845,2


In [33]:
clients.describe()

,age,taux,nbEnfantsAcharge
count,99807.000000,99806.000000,99800.000000
mean,43.733335,607.800503,1.245150
std,18.355480,336.279121,1.388624
min,-1.000000,-1.000000,-1.000000
25%,28.000000,420.000000,0.000000
50%,42.000000,520.000000,1.000000
75%,57.000000,827.000000,2.000000
max,84.000000,1399.000000,4.000000


### Gérer les valeurs manquantes

In [39]:
# Création d'un nouveau dataframe pour Imputation et Suppression
clients_imputed = clients.copy()
clients_dropped = clients.copy()

#### A-Remplacez les valeurs manquantes par mean or mode or median

In [40]:
# Remplacement des valeurs manquantes pour les colonnes numériques par la médiane
clients_imputed['age'] = clients_imputed['age'].fillna(clients_imputed['age'].median())
clients_imputed['taux'] = clients_imputed['taux'].fillna(clients_imputed['taux'].median())
clients_imputed['nbEnfantsAcharge'] = clients_imputed['nbEnfantsAcharge'].fillna(clients_imputed['nbEnfantsAcharge'].median())

In [41]:
# Remplacement des valeurs manquantes pour les colonnes catégorielles par la valeur la plus fréquente
clients_imputed['sexe'] = clients_imputed['sexe'].fillna(clients_imputed['sexe'].mode()[0])
clients_imputed['situationFamiliale'] = clients_imputed['situationFamiliale'].fillna(clients_imputed['situationFamiliale'].mode()[0])
clients_imputed['2eme voiture'] = clients_imputed['2eme voiture'].fillna(clients_imputed['2eme voiture'].mode()[0])

In [42]:
clients_imputed.describe(include='object')

,sexe,situationFamiliale,2eme voiture,immatriculation
count,100000,100000,100000,100000
unique,2,4,2,99989
top,M,En Couple,false,835 JQ 59
freq,70001,64174,87065,2


In [44]:
clients_imputed.describe()

,age,taux,nbEnfantsAcharge
count,100000.000000,100000.000000,100000.000000
mean,43.729990,607.630170,1.244660
std,18.337916,335.974983,1.387278
min,-1.000000,-1.000000,-1.000000
25%,28.000000,420.000000,0.000000
50%,42.000000,520.000000,1.000000
75%,57.000000,826.000000,2.000000
max,84.000000,1399.000000,4.000000


#### B-Supprimer les lignes avec des valeurs manquantes

In [50]:
# Pourcentage de valeurs manquantes
print(clients.isnull().sum() / len(clients) * 100)  

age                   0.193
sexe                  0.291
taux                  0.194
situationFamiliale    0.319
nbEnfantsAcharge      0.200
2eme voiture          0.220
immatriculation       0.000
dtype: float64


In [51]:
# Suppression des lignes avec des valeurs manquantes
clients_dropped = clients_dropped.dropna()

# Vérification des valeurs manquantes dans le nouveau dataframe
print(clients_dropped.isnull().sum())


age                   0
sexe                  0
taux                  0
situationFamiliale    0
nbEnfantsAcharge      0
2eme voiture          0
immatriculation       0
dtype: int64


### Sauvegarde des données nettoyées

In [55]:
# J'utilise le dataframe clients_imputed pour le premier essai
clients_imputed.to_csv('../data/processed/clients_cleaned.csv', index=False)
print("\nDonnées nettoyées sauvegardées dans 'clients_cleaned.csv'")


Données nettoyées sauvegardées dans 'clients_cleaned.csv'
